In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import torch
from tqdm import tqdm
from itertools import islice
from uuid import uuid4
from datasets import load_dataset
from dotenv import dotenv_values
from huggingface_hub import login

In [2]:
venv = dotenv_values('.env')
login(token=venv["HF_TOKEN"], add_to_git_credential=True)

# ASTRA_DB_API_ENDPOINT = venv["ASTRA_DB_API_ENDPOINT"]
# ASTRA_DB_APPLICATION_TOKEN = venv["ASTRA_DB_APPLICATION_TOKEN"]
# ASTRA_DB_NAMESPACE = venv["ASTRA_DB_NAMESPACE"]

MODEL_EMBED = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

Token has not been saved to git credential helper.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [3]:
# corpus_file = r"NaverLegal/corpus.csv"
dataset_name = "Zappu/Legal-vn"
collection_name = "AstraDB_Train_Legal_v2"

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_kwargs = {'device': device}
cache_dir = r"../.cache"

embeddings = HuggingFaceEmbeddings(
    model_name=MODEL_EMBED,
    cache_folder=cache_dir,
    model_kwargs=model_kwargs,
    show_progress=True,
)

/anaconda/envs/rag/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [7]:
dataset = load_dataset(dataset_name, "default", split="train")
dataset

Dataset({
    features: ['question', 'context', 'cid', 'qid'],
    num_rows: 79456
})

In [8]:
# def create_vector_store_from_dataset(dataset, collection_name, vector_store, chunk_size):
#     """
#     Create vectorstore from Huggingface Dataset

#     Args:
#         dataset_name: dataset from Hugging Face
#         collection_name: name of collection
#         vector_store: AstraDBVectorStore
#         chunk_size: size of chunk
#     """
    
#     for i in range(0, len(dataset), chunk_size):
#         # Lặp qua tất cả các split (train, test, validation)
#         documents = []
#         chunk = islice(dataset, i, i + chunk_size)
#         for row in tqdm(chunk, desc=f"Processing split {chunk}"):
#             text = row['context']
#             if isinstance(text, list):
#                  text = " ".join(text)  # Nối list thành string nếu cần
#             # Tiền xử lý text nếu cần
#             if '"' in text:
#                 text = text.replace('"', '')
#             if r'\n' in text:
#                 text = text.replace(r'\n', '')
#             if r'/' in text:
#                 text = text.replace(r'/', '')

#             doc = Document(
#                 page_content=text,
#                 metadata={"cid": row['cid']},
#             )
#             documents.append(doc)
#         uuids = [str(uuid4()) for _ in range(len(documents))]
#         print(f"Chunk: {i // chunk_size}/{len(dataset) // chunk_size + 1}")
#         vector_store.add_documents(documents=documents, ids=uuids)

#     print(f"Collection {collection_name} created successfully")

In [9]:
chunk_size = 1000
# create_vector_store_from_dataset(dataset, collection_name, vector_store, chunk_size)
for i in range(0, len(dataset), chunk_size):
    # Lặp qua tất cả các split (train, test, validation)
    documents = []
    chunk = islice(dataset, i, i + chunk_size)
    for row in tqdm(chunk, desc=f"Processing split {chunk}"):
        text = row['context']
        if isinstance(text, list):
                text = " ".join(text)  # Nối list thành string nếu cần
        # Tiền xử lý text nếu cần
        if '"' in text:
            text = text.replace('"', '')
        if r'\n' in text:
            text = text.replace(r'\n', '')
        if r'/' in text:
            text = text.replace(r'/', '')

        doc = Document(
            page_content=text,
            metadata={"cid": row['cid']},
        )
        documents.append(doc)
    uuids = [str(uuid4()) for _ in range(len(documents))]
    print(f"Chunk: {i // chunk_size}/{len(dataset) // chunk_size + 1}")
    vector_store.add_documents(documents=documents, ids=uuids)

print("Done")

Processing split <itertools.islice object at 0x7fab705131a0>: 1000it [00:00, 20991.04it/s]

Chunk: 0/80


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Processing split <itertools.islice object at 0x7fab2e6cfa60>: 1000it [00:00, 12259.32it/s]

Chunk: 1/80


...

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Processing split <itertools.islice object at 0x7fab029ef510>: 456it [00:02, 157.85it/s]

Chunk: 79/80


Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Done


In [11]:
faiss_path = "VectorStore/Train_Legal_v1.faiss"

In [ ]:

vector_store.save_local(faiss_path)
print(f"Collection {collection_name} created successfully")

Collection AstraDB_Train_Legal_v2 created successfully


In [ ]:
import ray, time
ray.init()

2024-11-25 07:04:24,035	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-25 07:04:24,059	INFO worker.py:1634 -- Connecting to existing Ray cluster at address: 10.0.0.4:6379...
2024-11-25 07:04:24,068	INFO worker.py:1810 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.11
Ray version:,2.39.0
Dashboard:,http://127.0.0.1:8265


In [14]:
dataset = load_dataset(dataset_name, "default", split="train")
dataset

Dataset({
    features: ['question', 'context', 'cid', 'qid'],
    num_rows: 79456
})

In [ ]:
small_dataset = dataset.select(range(79456))
small_dataset

Dataset({
    features: ['question', 'context', 'cid', 'qid'],
    num_rows: 800
})

In [ ]:
@ray.remote(num_cpus=1, num_gpus=0)
def predict_by_chunk(i, chunk, vtstore_ref):
    start_time = time.time()
    predict_path = f"Predicts/predict_{i}.txt"
    with open(predict_path, 'w') as p:
        for row in tqdm(chunk, desc=f"Processing split {chunk} {i}"):
            content = row['question']
            qid = row['qid']

            results = vtstore_ref.similarity_search(
                content,
                k = 10,
            )
            cids = [doc.metadata['cid'] for doc in results]  # Assuming 'qid' is what you meant by cid
            more_cid = ""
            for i in cids:
                more_cid += f" {i[1:-1]}"
            response = f"{qid} {more_cid}"

            # response = similarity_search(content, faiss_path, qid)
            p.write(response + '\n')
    return f"processed {i} chunk: " + "{:.4f}".format(time.time() - start_time)    

In [18]:
vtstore = FAISS.load_local(faiss_path, embeddings, allow_dangerous_deserialization=True)
vtstore_ref = ray.put(vtstore)

In [ ]:
chunk_size = 1000
cnt = 0
chunk_results = []
max_task = 8
total_chunk = int(len(small_dataset) // chunk_size)
for i in range(0, total_chunk):
    cnt += 1
    chunk = small_dataset.shard(num_shards=total_chunk, index=i)
    while cnt - 1 >= max_task:
        ready_ids, _ = ray.wait(chunk_results, num_returns=1)
        ready_id = ready_ids[0]
        chunk_results.remove(ready_id)
        cnt -= 1
    chunk_results.append(predict_by_chunk.remote(i, chunk, vtstore_ref))
processed_chunks = ray.get(chunk_results)

Processing split Dataset({=35384) 
(predict_by_chunk pid=35384)     features: ['question', 'context', 'cid', 'qid'],
(predict_by_chunk pid=35384)     num_rows: 100
Batches:   0%|          | 0/1 [00:00<?, ?it/s]
(predict_by_chunk pid=35386) 
(predict_by_chunk pid=35382) 
(predict_by_chunk pid=35382) }) 0:   1%|          | 1/100 [00:01<03:15,  1.97s/it]
(predict_by_chunk pid=35384) 
(predict_by_chunk pid=35381) 
(predict_by_chunk pid=35386) 
Batches: 100%|██████████| 1/1 [00:00<00:00, 20.20it/s]
(predict_by_chunk pid=35382) 
Batches: 100%|██████████| 1/1 [00:00<00:00, 22.06it/s]
(predict_by_chunk pid=35384) 
(predict_by_chunk pid=35381) 
(predict_by_chunk pid=35386) 
(predict_by_chunk pid=35382) 
(predict_by_chunk pid=35384) 
(predict_by_chunk pid=35381) 
(predict_by_chunk pid=35386) 
(predict_by_chunk pid=35386) 
(predict_by_chunk pid=35382) 
(predict_by_chunk pid=35384) 
(predict_by_chunk pid=35384) 
(predict_by_chunk pid=35381) 
(predict_by_chunk pid=35386) 
(predict_by_chunk pid=3538

In [22]:
print(processed_chunks)

['processed [73066] chunk: 8.3650', 'processed [63558 63559 63560] chunk: 8.6792', 'processed [167983] chunk: 8.4100', 'processed [63777] chunk: 8.5137', 'processed [70726] chunk: 4.5242', 'processed [81730] chunk: 4.4061', 'processed [192983] chunk: 5.1766', 'processed [206691] chunk: 5.3145']


In [20]:
ray.shutdown()